In [61]:
import pandas as pd
import glob
import os
import shutil


In [62]:
f = open('findMasterReport.txt', 'w')
root = ".\Raw"
for functionTypeDirectory in os.listdir(root):
    f.write("\nfunctionTypeDirectory: "+functionTypeDirectory)
    if os.path.isdir(os.path.join(root, functionTypeDirectory)):
        for functionDirectory in os.listdir(os.path.join(root, functionTypeDirectory)):
            f.write("\nfunctionDirectory: "+functionDirectory)
            for numberOfNodesDirectory in os.listdir(os.path.join(root, functionTypeDirectory, functionDirectory)):
                f.write("\nnumberOfNodesDirectory: "+numberOfNodesDirectory)
                destinationPath =  os.path.join(root, functionTypeDirectory, functionDirectory, numberOfNodesDirectory, 'source-data')
                for nodeDirectory in os.listdir(destinationPath):
                    path =  os.path.join(destinationPath, nodeDirectory)
                    f.write("\nnodeDirectory: "+nodeDirectory + "\n")
                    all_files = glob.glob(path + "/*.csv")
                    dfs = []
                    i = 1
                    for filename in all_files:
                        df = pd.read_csv(filename)
                        dfs.append(df)
                        if not os.path.exists(os.path.join(destinationPath.replace("Raw", "Organised"), str(i))):
                            os.makedirs(os.path.join(destinationPath.replace("Raw", "Organised"), str(i)))
                        shutil.copyfile(filename, os.path.join(destinationPath.replace("Raw", "Organised"), str(i), os.path.basename(filename)))

                        i += 1
                    i = 1
                    for df in dfs:
                        f.write("Experiment number:" + str(i))
                        f.write(" CPU Mean:"+ str(df['CPU'].mean()))
                        f.write(" CPU Max:" + str(df['CPU'].max()))
                        f.write(" RAM Mean:"+ str(df['RAM'].mean()))
                        f.write(" RAM Max:" + str(df['RAM'].max()) + "\n")
                        i += 1

f.close()
